# Extracting Mine Boundary with Segment Geospatial

## Overview

We use the [samgeo](https://samgeo.gishub.org/) package to segment a GeoTIFF image of a mining area. To ensure we get only the mine boundary, we will prompt the model with a set of 'foreground' and 'background' points.

The image used here is a RGB composite of a mining area derived from a Sentinel-2 image. The image has been exported from Google Earth Engine. See [reference script](https://code.earthengine.google.co.in/54fc87178f1a0cc31e06b9314b14428f).

In [ ]:
%%capture
if 'google.colab' in str(get_ipython()):
    !pip install segment-geospatial

In [ ]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

In [ ]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

data_url = 'https://storage.googleapis.com/spatialthoughts-public-data/s2/'

image = 's2_mines_rgb.tif'

download(data_url + image)


In [ ]:
image_path = os.path.join(data_folder, image)

m = leafmap.Map(width=800, height=500)
m.add_raster(image_path, layer_name="Image")
m

## Segmenting the Image

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    sam_kwargs=None,
)

### Interactive Segmentation

In [ ]:
mask = 'segment_interactive.tif'
interactive_mask_path = os.path.join(output_folder, mask)
vector = 'segment_interactive.gpkg'
interactive_vector_path = os.path.join(output_folder, vector)

In [ ]:
sam.set_image(image_path)
sam.show_map()

In [ ]:
sam.save_prediction(interactive_mask_path)
sam.tiff_to_gpkg(interactive_mask_path, interactive_vector_path, simplify_tolerance=None)

## Segmentation with Training Samples

In [ ]:
mask = 'segment_training.tif'
training_mask_path = os.path.join(output_folder, mask)
vector = 'segment_training.gpkg'
training_vector_path = os.path.join(output_folder, vector)

In [ ]:
foreground_coords = [
    [-105.2030365777389, 43.59243279130138],
    [-105.31799690813952, 43.58165243450691],
    [-105.31389118205367, 43.65410388290101],
    [-105.33134051791814, 43.69753284641334],
    [-105.29130968858229, 43.75501854039891],
    [-105.31127393080612, 43.72971279039413],
    [-105.34977689826118, 43.74965309055227]
]

background_coords = [
    [-105.27024271682525, 43.59987910041002],
    [-105.24252906574645, 43.72165914628937],
    [-105.22507972988198, 43.64334727429318],
    [-105.32310394017892, 43.609913006905344],
    [-105.38109732113958, 43.74094305344275]
]

n_foreground_pts = len(foreground_coords)
n_background_pts = len(background_coords)


point_coords = foreground_coords + background_coords

point_labels = [1] * n_foreground_pts + [0] * n_background_pts

sam.set_image(image_path)
sam.predict(point_coords=point_coords, point_labels=point_labels, point_crs="EPSG:4326", output=training_mask_path)


In [ ]:
m.add_raster(training_mask_path, layer_name="segments", nodata=0, cmap="Greens", opacity=1)
m

## Polygonize the raster data

Save the segmentation results as a geopackage

In [ ]:
sam.tiff_to_gpkg(training_mask_path, training_vector_path, simplify_tolerance=None)

In [28]:
m.add_vector(training_vector_path, layer_name='Polygons')
m

Map(bottom=766231.0, center=[43.576778000000004, -105.31012949999999], controls=(ZoomControl(options=['positio…